In [3]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import FileProcess as fp


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


('Total Words in DataSet:', 400000)
('shape of data:', (866842, 5))
Started
('time-taken:', 9.5367431640625e-07)


In [5]:
print (fp.window_size)
batch_size=10
vocabulary_size=400000
embedding_size=200
embedding_filename = "glove_6B_200d.txt"
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]


2


In [6]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]

#print(fp.get_sentences(files[0]))

[' background  1 the applicant is a citizen of the peoples republic of china (prc).', 'he arrived in australia on 25 december 2002.', 'on 14 january 2003 the applicant applied for a protection (class xa) visa.', 'on 18 june 2003 a delegate of the minister for immigration and multicultural affairs (as he was then) refused to grant the applicant a protection (class xa) visa ("delegate\'s decision").', "on 7 july 2003 the applicant applied to the refugee review tribunal for review of the delegate's decision.", 'on 5 may 2004 the applicant gave oral evidence at a tribunal hearing.', "on 3 march 2005 the tribunal made a decision affirming the delegate's decision.", 'on 23 march 2006 federal magistrate lloyd-jones made orders by consent remitting the matter to the tribunal.', 'on 24 may 2006 the tribunal, as reconstituted, sent a letter, by facsimile, inviting the applicant to attend a hearing.', 'on 26 may 2006 the tribunal received a "response to hearing invitation" form indicating that th

In [7]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();

d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
#print(docvec)

[-2.7762175  -0.45936382 -0.618449   -0.8609642   0.12776479  0.7008039
 -0.18298188 -0.3065114  -1.0207447   0.44748068 -0.5251434  -0.6478685
  0.07363328 -0.9913516  -0.08037782  0.3569275   0.04830581 -0.3157292
 -0.33566946 -1.8007975  -0.6615384  -0.45662943  0.35856095 -1.6224141
  1.7292947   0.01610192  0.15269798 -0.2633624  -0.42103472  0.4310425
 -0.8186927  -1.817319   -1.9041965  -0.8068781  -1.777631   -1.8509216
 -0.4281155   0.4977019   0.61602515  0.76634425  1.3259608  -0.31260923
 -1.3816838  -0.37780055 -1.1810374  -1.6388259   1.0663178   0.06534389
 -0.7727587  -0.11205402 -1.9782453   0.08237717  1.9480875  -1.8146058
 -1.3543537   1.3657124  -0.25756338  0.54493374 -0.6701863   0.32007536
  0.05760738  1.294205   -0.32358631 -0.9725704  -1.0321938  -0.39936873
  0.15072757 -0.7044433  -1.2028613  -0.57092947 -1.0215961   2.1046119
 -0.02798054 -0.17244416  0.8805654   0.5527702   1.502657   -2.0060904
  0.67571783  1.7193362   1.0933233  -1.2774023   0.8924701 

In [8]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [9]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [10]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

# words_data=pd.read_csv("words_data.csv");
# catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

('Total Words in DataSet:', 400000)
Loaded embeddings
reading data completed


In [11]:
#### TO avoid downloading tf hub modules everytime
##  export TFHUB_CACHE_DIR=/usr/local/bin

#sen2vec embeddings
def sen2vec1(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

    
def sent2vector(sent):
    words = word_tokenize(sent.lower())
    emb=[]
    for w in words:
        if w in embedding_array.keys():
            emb.append(embedding_array.get(w));
        else:
            emb.append(embedding_array.get('unk'));
    return np.mean(np.array(emb),axis=0)

def get_sent_embeddings(sent_list):
    out_vec=np.zeros((len(sent_list)-1,200))
    for i  in range(len(sent_list)-1):
        out_vec[i]=sent2vector(sent_list[i]);
    return out_vec
    
# sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

In [12]:
def get_statements(file):
    #with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
    with open(dirpath + file, 'r') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    #catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    #sentences+=tokenizer.tokenize(sentence.text)
    
    
    #sentences = [w for w in sentences if not w in stop_words_list] 
    #catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases

In [15]:
sentences,catchphrases=get_statements(files[0])
start=time.time()
sentences, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
print(sentences,catch_words)
print("time-taken:",time.time()-start)

(        Id     file_id  is_catchword                                words
0       s0  07_209.xml             0            [2093, 176, 0, 19067, 14]
1       s0  07_209.xml             1               [176, 0, 19067, 14, 7]
2       s0  07_209.xml             0              [0, 19067, 14, 7, 3941]
3       s0  07_209.xml             0              [19067, 14, 7, 3941, 3]
4       s0  07_209.xml             0                  [14, 7, 3941, 3, 0]
5       s0  07_209.xml             0                [7, 3941, 3, 0, 5527]
6       s0  07_209.xml             0              [3941, 3, 0, 5527, 874]
7       s0  07_209.xml             0                 [3, 0, 5527, 874, 3]
8       s0  07_209.xml             0               [0, 5527, 874, 3, 132]
9       s0  07_209.xml             0          [5527, 874, 3, 132, 201534]
10      s1  07_209.xml             0               [18, 1149, 6, 603, 13]
11      s1  07_209.xml             0              [1149, 6, 603, 13, 579]
12      s1  07_209.xml             0 

In [13]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [14]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [15]:
def get_file_sentences(file_name):
    temp_data=full_data[(full_data.file_id==file_name) & full_data['Id'].str.startswith(('s'))]
    return list(temp_data.text.values)[:-1]

In [16]:
import time

class CatchPhraseExtractor(object):

    def __init__(self, graph):
        self.total_loss=0.0
        self.build_graph(graph)

    def build_graph(self, graph):
        print("generating graph");

        with graph.as_default():            
            self.phrases_indices = tf.placeholder(tf.int32, shape=[None,5])
            self.phrase_labels = tf.placeholder(tf.float32, shape=[None,1])
            
            #W = tf.Variable(embedding_array, dtype=tf.float32)
            W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            phrase_tensor = tf.nn.embedding_lookup(W, self.phrases_indices)
            
            #self.sentence_indices = tf.placeholder(tf.int32, shape=[None,1])
            #self.sentence_embeddings = tf.placeholder(tf.float32, shape=[None,200])
            #sentence_tensor = tf.nn.embedding_lookup(self.sentence_embeddings, self.sentence_indices)
            #sentence_tensor=tf.reshape(sentence_tensor, [tf.shape(sentence_tensor)[0], 200])
                                                     
            self.doc_embedding = tf.placeholder(tf.float32, shape=[1,200])
            document_tensor = tf.tile(self.doc_embedding, [tf.shape(self.phrases_indices)[0], 1])
            
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], 1000])
            conv_filter = tf.Variable(tf.truncated_normal((1000, 200), stddev=0.1));
            word_tensor = tf.nn.relu(tf.matmul(phrase_tensor, conv_filter));
            
            #stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
            stacked_tensor = tf.concat(values=[word_tensor, document_tensor], axis=1)
            weights_input = tf.Variable(tf.random_normal((400, 100), stddev=0.2));
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2));
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2));
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2));
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2);
            
            #prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.phrase_labels,logits=self.sentence_values)
            #self.loss = tf.reduce_mean(prediction_loss)
            self.loss=tf.reduce_mean(tf.keras.metrics.mean_squared_error(self.phrase_labels, self.sentence_values))
            #self.loss=tf.convert_to_tensor(tf.reduce_mean(self.sentence_values));
            
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(0.001)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)
            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        print ('#######h1 after mul: ', h1.shape)
        #h1 = tf.add(h1, bias1)
        #print ('#######h1_afterbias: ', h1.shape)
        h1 = tf.nn.tanh(h1)
        print ('#############h1.shape', h1.shape)
        print ('###########weights_input_2.shape', weights_input_2.shape)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        #h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        #h2 = tf.round(tf.transpose(h2))
        return tf.transpose(h2)
        #return tf.cast(tf.to_float(h2>0.5),dtype=tf.float32)
    
    
    '''
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
    '''     
                                                   
    def train(self, sess, num_steps):
        self.init.run()
        #print("Initailized")
        start=time.time()
        total_loss=0.0;
        for file in files[:20]:
            start=time.time()
            sentences, catchphrases=get_statements(file);
            file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
            #print("Time taken:",time.time()-start)
            
            #needs sentences list
            #sentences_list = fp.get_sentences(file) #indexes/order preserved
            #sentence_embeddings = sen2vec(sentences_list)
            #sentence_embeddings = get_sent_embeddings(sentences_list)
            
            #sentence_ids_to_indices = np.array([int(x[1:]) for x in list(file_data_frame['Id'].values)])
            
            #phrase inputs and labels
            phrases_indices = np.array(list(file_data_frame['words'].values))
            phrases_label_matrix = np.array(list(file_data_frame['is_catchword'].values))
            
            
            #doc embedding
            doc_embedding = d2v_model.docvecs[file]

            
            doc_embedding = np.reshape(doc_embedding, (1, 200))
            #sentence_ids_to_indices = np.reshape(sentence_ids_to_indices, (sentence_ids_to_indices.shape[0], 1))
            phrases_label_matrix = np.reshape(phrases_label_matrix, (phrases_label_matrix.shape[0], 1))
            
            '''
            print ('*************Doc: ', doc_embedding.shape)
            print ('*************phrases_indices: ', phrases_indices.shape)
            print ('*************sentence_ids_to_indices: ', sentence_ids_to_indices.shape)
            print ('*************phrases_label_matrix: ', phrases_label_matrix.shape)
            print ('*************sentence_embeddings: ', sentence_embeddings.shape)           
            
            feed_dict = {self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix, 
                         self.sentence_indices:sentence_ids_to_indices, 
                         self.sentence_embeddings: sentence_embeddings}'''
            
            feed_dict = {self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix}
            
            #print("sending inputs");
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            print('Train Finished for ', file, ' timetaken: ', time.time()-start,' loss_val=', loss_val)
            #for i in range(len(outputs)):
            #    print(outputs[i])
            
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [17]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [18]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph)
start=time.time()

#print(fp.get_dataframe(files[0]))
#sentences_list = fp.get_file_sentences(files[0]) #indexes/order preserved
#sentence_embeddings = sen2vec(sentences_list)
end=time.time()
print("time taken:",end-start);

generating graph
('#######h1 after mul: ', TensorShape([Dimension(100), Dimension(None)]))
('#############h1.shape', TensorShape([Dimension(100), Dimension(None)]))
('###########weights_input_2.shape', TensorShape([Dimension(100), Dimension(1)]))
('time taken:', 2.5033950805664062e-05)


In [ ]:
with tf.Session(graph=graph) as sess:
  
    #model.load_embeddings(embedding_array)    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    #global_step = tf.train.get_or_create_global_step()
    for i in range(20):
        model.train(sess, 20)
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

('Train Finished for ', '07_209.xml', ' timetaken: ', 2.198723077774048, ' loss_val=', 0.50694317)
('Train Finished for ', '06_159.xml', ' timetaken: ', 0.5551731586456299, ' loss_val=', 0.20376645)
('Train Finished for ', '09_245.xml', ' timetaken: ', 13.486531019210815, ' loss_val=', 0.21002378)
('Train Finished for ', '09_636.xml', ' timetaken: ', 1.794409990310669, ' loss_val=', 0.57115394)
('Train Finished for ', '06_460.xml', ' timetaken: ', 17.61236310005188, ' loss_val=', 0.18755013)
('Train Finished for ', '06_1396.xml', ' timetaken: ', 2.1055359840393066, ' loss_val=', 0.2705482)
('Train Finished for ', '07_1402.xml', ' timetaken: ', 16.059673070907593, ' loss_val=', 0.1818798)
('Train Finished for ', '06_713.xml', ' timetaken: ', 0.774461030960083, ' loss_val=', 0.14615428)
('Train Finished for ', '06_1260.xml', ' timetaken: ', 2.5985100269317627, ' loss_val=', 0.5840475)
('Train Finished for ', '08_711.xml', ' timetaken: ', 2.3841660022735596, ' loss_val=', 0.36740142)
('Tr

('Train Finished for ', '09_636.xml', ' timetaken: ', 1.8062479496002197, ' loss_val=', 0.41095674)
('Train Finished for ', '06_460.xml', ' timetaken: ', 17.715312957763672, ' loss_val=', 0.31818762)
('Train Finished for ', '06_1396.xml', ' timetaken: ', 2.1083879470825195, ' loss_val=', 0.19226667)
('Train Finished for ', '07_1402.xml', ' timetaken: ', 15.983786821365356, ' loss_val=', 0.06748198)
('Train Finished for ', '06_713.xml', ' timetaken: ', 0.7511749267578125, ' loss_val=', 0.46147192)
('Train Finished for ', '06_1260.xml', ' timetaken: ', 2.6322031021118164, ' loss_val=', 0.10465203)
('Train Finished for ', '08_711.xml', ' timetaken: ', 2.401757001876831, ' loss_val=', 0.09160807)
('Train Finished for ', '08_1030.xml', ' timetaken: ', 0.9998109340667725, ' loss_val=', 0.506294)
('Train Finished for ', '06_707.xml', ' timetaken: ', 1.6326320171356201, ' loss_val=', 0.3337362)
('Train Finished for ', '06_1326.xml', ' timetaken: ', 7.644528865814209, ' loss_val=', 0.17799397)


('Train Finished for ', '06_1396.xml', ' timetaken: ', 2.294508934020996, ' loss_val=', 0.4519029)
